In [1]:
train_dir_LandSAT = r"C:\Users\HPCL\OneDrive - University of New Orleans\Documents\Research\Year 1\Pytorch\Year1_Research\New Experimets\Data\Landsat 30m Resolution"
train_dir_Sentinel = r"C:\Users\HPCL\OneDrive - University of New Orleans\Documents\Research\Year 1\Pytorch\Year1_Research\New Experimets\Data\Sentinel uint16 Data 0%water and 0%land exclude"
save_last = r"C:\Users\HPCL\OneDrive - University of New Orleans (1)\Alsamman Research\HPC_Results\Saiful_Results_1"


In [2]:
import time

today=str(time.ctime().replace(":","-").replace(" ","-"))
print(today)

save_last = save_last+"\\"+today
    
print(save_last)

Fri-Apr-14-11-03-54-2023
C:\Users\HPCL\OneDrive - University of New Orleans (1)\Alsamman Research\HPC_Results\Saiful_Results_1\Fri-Apr-14-11-03-54-2023


In [3]:
import os



if not os.path.exists(save_last):
    os.makedirs(save_last) 
    
    
    

In [4]:
import numpy as np
import matplotlib.pyplot as plt
# import os
import copy
import sys
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from PIL import Image
from collections import OrderedDict
import pandas as pd
from tqdm import tqdm
import random
from collections import defaultdict 
import tifffile as tiff
import imageio as io
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

import segmentation_models_pytorch as smp
from skimage.morphology import binary_dilation, binary_erosion
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from sklearn.metrics import average_precision_score

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# Loss Functions

In [6]:
def weighted_bce_loss(output,y,weight):
    epsilon= 1e-7
    output = torch.clamp(output, epsilon, 1.-epsilon)
    logit_output = torch.log(output/(1.-output))
    
    loss = (1.-y)*logit_output + (1.+(weight-1.)*y) * (torch.log(1.+torch.exp(-torch.abs(logit_output))) + torch.maximum(-logit_output,torch.tensor(0.).cuda()))
    return torch.sum(loss)/torch.sum(weight)

def weighted_dice_loss(output,y,weight):
    smooth = 1.
    w,m1,m2 = weight*weight, y, output
    intersection = (m1*m2)
    score = (2.*torch.sum(w*intersection)+smooth)/(torch.sum(w*m1)+torch.sum(w*m2)+smooth)
    loss  = 1.-torch.sum(score)
    return loss

def iou_loss(y_pred, y_true, weight):
    weight = weight*weight
    intersection = y_true * y_pred
    not_true     = 1 - y_true
    union        = y_true + (not_true * y_pred)
    iou          = (torch.sum(intersection * weight)) / (torch.sum(union * weight))

    loss = 1-iou
    return loss
    

def minkowskySum(y_pred, y_true):
    sumDif = y_pred+y_true 
    sumTrue = (2*y_true)
    
    sumDif = sumDif - sumTrue
    
    return sumDif.sum()/sumTrue.sum()

def huberLoss(output, target):
    delta= 1.0
    
    diff = output - target
    loss = torch.where(torch.abs(diff) < delta, 0.5 * torch.pow(diff,2), delta * torch.abs(diff)-0.5*delta**2)
    
    loss = torch.mean(loss)
    
    return loss
    
def triplet_Loss(y_pred, y_true,negative):
    margin = 0.2
    
    anchor = y_pred
    positive = y_true
    
   
    
    # compute the distance between the anchor and positive embeddings
    pos_dist = torch.norm(anchor - positive,p=2,dim=1)
    
    # compute the distance between the anchor and negative embeddings
    neg_dist = torch.norm(anchor - negative,p=2, dim=1)
    
    #compute the triplet loss
    loss = torch.mean(torch.maximum(pos_dist - neg_dist + margin, torch.tensor(0.0)))
    
    return loss
    
    
def border_loss(output,y,pool_size=(9,9), pad=(4,4)):
    y      = y.type(torch.float32)
    
    output = output.type(torch.float32)
    
    averaged_mask = F.avg_pool2d(y,kernel_size=pool_size,stride=(1,1), padding=pad)
    border = (averaged_mask>0.005).type(torch.float32) * (averaged_mask<0.995).type(torch.float32)
    weight = torch.ones_like(averaged_mask)
    w0     = torch.sum(weight)
    weight+= border*2
    w1     = torch.sum(weight)
    weight*= (w0/w1)
    loss   = weighted_bce_loss(output,y,weight) + weighted_dice_loss(output,y,weight) + iou_loss(output,y,weight)# + tripletLoss(output, y)
#     mLoss = tripletLoss(output, y)
    return loss

# Functions

In [7]:
################ Metrics #######################
def IoU_pr_rec_f1(y_true, y_pred):
    
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)
    y_pred = ((y_pred)*1.).type(torch.float32)
    
    tp = torch.sum(y_true*(y_pred))
    tn = torch.sum((1-y_true)*((1-y_pred)))
    fp = torch.sum((1-y_true)*(y_pred))
    fn = torch.sum((y_true)*((1-y_pred)))
    
    pr  = (tp/(tp+fp))
    rec = (tp/(tp+fn))
    f1  = ((2*pr*rec)/(pr+rec))
    tnr = (tn/(tn+fp))
    fpr = (fp/(fp+tn))
    
    intersection = y_true*y_pred
    not_true     = 1 - y_true
    union        = y_true + (not_true * y_pred)
    iou         = (torch.sum(intersection)) / (torch.sum(union))
    
    return iou, pr, rec, f1, tnr, fpr

# Saving Metrics
def metrics():
    x = np.arange(0,1,0.05)
    IoU_      = []
    threshold = []
    precision = []
    recall    = []
    F_score   = []
    mF_score  = []
    TNR       = []
    FPR       = []
    name_list = []

    dict_1 = {'Threshold': threshold,
              'Name':name_list,
              'IoU':IoU_,
              'Precision':precision,
              'Recall':recall,
              'F-Score':F_score,
              'mF-Score':mF_score,
              'True Negative Rate':TNR,
              'False Positive Rate':FPR}
    return dict_1

def best_f_score(name, test_masks, predictions) :
    dict_1 = metrics()
    y = 0
    outer = 0
    check = 0
    x = 0 
    y = 1
    while outer<3:    
        if y>1:
            m = y-1
            y-= m
        z = np.linspace(x, y, 21)
        for i in z:
#             print(i)
            y_true = torch.from_numpy(test_masks)
            y_pred = torch.from_numpy((predictions>i)*1)

            tp = torch.sum(y_true*(y_pred),dim=[1,2,3])
            tn = torch.sum((1-y_true)*((1-y_pred)),dim=[1,2,3])
            fp = torch.sum((1-y_true)*(y_pred),dim=[1,2,3])
            fn = torch.sum((y_true)*((1-y_pred)),dim=[1,2,3])

            pr  = (tp/(tp+fp))
            rec = (tp/(tp+fn))
            score  = ((2*pr*rec)/(pr+rec))
            idx    = torch.isnan(score)
            score[idx] = 0
            score  = torch.sum(score)/len(X_test)
            
            a,b,c,d,e,f = IoU_pr_rec_f1(torch.from_numpy(test_masks), torch.from_numpy(predictions>i))
            dict_1['IoU'].append(a.numpy())
            dict_1['Threshold'].append(i)
            dict_1['Precision'].append(b.numpy())
            dict_1['Recall'].append(c.numpy())
            dict_1['F-Score'].append(d.numpy())
            dict_1['mF-Score'].append(score.cpu().detach().numpy())
            dict_1['True Negative Rate'].append(e.numpy())
            dict_1['False Positive Rate'].append(f.numpy())
            dict_1['Name'].append(name)
            if d>check:
                check = d
                x = i
            else:
                pass
        if outer == 0:
            y = x+0.1
        elif outer==1:
            y = x+0.01
        outer+=1
    
    df = pd.DataFrame(dict_1)
    df = df.sort_values(by=['F-Score'], ascending=False)
    df = df.iloc[0:1]
    
    AP = average_precision_score(test_masks.reshape(-1), predictions.reshape(-1))
    df['AP'] = AP
        
    return df

dict_1 = metrics()

# Load Labeled Data

In [8]:
X_train_LandSAT = np.load(r"{}\X_train_4500_30m_res.npy".format(train_dir_LandSAT))
X_val_LandSAT   = np.load(r"{}\X_val_500_30m_res.npy".format(train_dir_LandSAT))
X_test_LandSAT  = np.load(r"{}\X_test_2000_30m_res.npy".format(train_dir_LandSAT))

Y_train_LandSAT = (X_train_LandSAT[...,1]-X_train_LandSAT[...,3])/(X_train_LandSAT[...,1]+X_train_LandSAT[...,3])
Y_val_LandSAT   = (X_val_LandSAT[...,1]-X_val_LandSAT[...,3])/(X_val_LandSAT[...,1]+X_val_LandSAT[...,3])
Y_test_LandSAT  = (X_test_LandSAT[...,1]-X_test_LandSAT[...,3])/(X_test_LandSAT[...,1]+X_test_LandSAT[...,3])

Y_train_LandSAT = ((Y_train_LandSAT<1.)*1).astype('float32')
Y_val_LandSAT   = ((Y_val_LandSAT<1.)*1).astype('float32')
Y_test_LandSAT  = ((Y_test_LandSAT<1.)*1).astype('float32')

X_train_LandSAT = X_train_LandSAT[...,2::-1].copy()
X_val_LandSAT   = X_val_LandSAT[...,2::-1].copy()
X_test_LandSAT  = X_test_LandSAT[...,2::-1].copy()

X_train_LandSAT = X_train_LandSAT - X_train_LandSAT.min(axis=(1,2), keepdims=True) 
X_val_LandSAT   = X_val_LandSAT  - X_val_LandSAT.min(axis=(1,2), keepdims=True)
X_test_LandSAT  = X_test_LandSAT - X_test_LandSAT.min(axis=(1,2), keepdims=True) 

X_train_LandSAT = X_train_LandSAT / X_train_LandSAT.max(axis=(1,2), keepdims=True) 
X_val_LandSAT   = X_val_LandSAT  / X_val_LandSAT.max(axis=(1,2), keepdims=True)
X_test_LandSAT  = X_test_LandSAT / X_test_LandSAT.max(axis=(1,2), keepdims=True) 

C:\Users\HPCL\AppData\Local\Temp\ipykernel_22328\3036067045.py:5: RuntimeWarning: invalid value encountered in true_divide
  Y_train_LandSAT = (X_train_LandSAT[...,1]-X_train_LandSAT[...,3])/(X_train_LandSAT[...,1]+X_train_LandSAT[...,3])
C:\Users\HPCL\AppData\Local\Temp\ipykernel_22328\3036067045.py:7: RuntimeWarning: invalid value encountered in true_divide
  Y_test_LandSAT  = (X_test_LandSAT[...,1]-X_test_LandSAT[...,3])/(X_test_LandSAT[...,1]+X_test_LandSAT[...,3])


In [9]:
## SENTINEL DATA

In [10]:
np.random.seed(49)
pop = np.arange(5121)
sample_train = np.random.choice(pop, 4608, replace=False)
sample_val  = np.delete(pop, sample_train)
len(sample_train), len(sample_val)

(4608, 513)

In [11]:
X_sen      = np.load(r"{}\X_train_sentinel_6_channles_5121.npy".format(train_dir_Sentinel))
X_test_sen = np.load(r"{}\X_test_sentinel_6_channles_2053.npy".format(train_dir_Sentinel))
X_train_sen= X_sen[sample_train]
X_val_sen  = X_sen[sample_val]

Y_train_sen         = (X_train_sen[...,1]-X_train_sen[...,3])/(X_train_sen[...,1]+X_train_sen[...,3])
Y_val_sen           = (X_val_sen[...,1]-X_val_sen[...,3])/(X_val_sen[...,1]+X_val_sen[...,3])
Y_test_sen         = (X_test_sen[...,1]-X_test_sen[...,3])/(X_test_sen[...,1]+X_test_sen[...,3])

Y_train_sen         = ((Y_train_sen<1.)*1).astype('float32')
Y_val_sen           = ((Y_val_sen<1.)*1).astype('float32')
Y_test_sen          = ((Y_test_sen<1.)*1).astype('float32')

X_train_sen         = X_train_sen[...,2::-1].copy()
X_val_sen           = X_val_sen[...,2::-1].copy()
X_test_sen          = X_test_sen[...,2::-1].copy()

X_train_sen         = np.clip(X_train_sen, 0, 3558) 
X_val_sen           = np.clip(X_val_sen, 0, 3558)
X_test_sen          = np.clip(X_test_sen, 0, 3558) 

X_train_sen         = (X_train_sen / 3558).astype(np.float32) 
X_val_sen           = (X_val_sen  / 3558).astype(np.float32)
X_test_sen          = (X_test_sen / 3558).astype(np.float32) 

Y_train_sen.shape

(4608, 128, 128)

In [12]:
Y_test= Y_test_sen # np.concatenate((Y_test_LandSAT,Y_test_sen),axis = 0)
Y_test.shape

(2053, 128, 128)

In [13]:
# Contours for Sentinel

In [ ]:
Y_train_sen  = np.array([binary_dilation(binary_dilation(mask))-mask for mask in Y_train_sen], dtype='float64')
Y_val_sen   = np.array([binary_dilation(binary_dilation(mask))-mask for mask in Y_val_sen], dtype='float64')
Y_test_sen  = np.array([binary_dilation(binary_dilation(mask))-mask for mask in Y_test_sen], dtype='float64')

In [ ]:
from sklearn.utils import shuffle
X_sen, Y_sen = shuffle(X_train_sen, Y_train_sen, random_state=0)

In [ ]:
plt.imshow(X_sen[0]+Y_sen[0,...,np.newaxis])

In [ ]:
# Contours for LandSAT

In [ ]:
Y_train_LandSAT  = np.array([binary_dilation(mask)-mask for mask in Y_train_LandSAT], dtype='float64')
Y_val_LandSAT   = np.array([binary_dilation(mask)-mask for mask in Y_val_LandSAT], dtype='float64')
Y_test_LandSAT  = np.array([binary_dilation(mask)-mask for mask in Y_test_LandSAT], dtype='float64')

In [ ]:
X_LS, Y_LS = shuffle(X_train_LandSAT, Y_train_LandSAT, random_state=0)

In [ ]:
plt.imshow(X_LS[0]+Y_LS[0,...,np.newaxis])

In [ ]:
# Concatenate the inputs and labels#

In [ ]:
X_train = np.concatenate((X_LS,X_sen),axis = 0)
print(X_train.shape)

X_val = np.concatenate((X_val_LandSAT,X_val_sen),axis = 0)
print(X_val.shape)

X_test = np.concatenate((X_test_LandSAT,X_test_sen),axis = 0)
print(X_test.shape)

In [ ]:
Y_train=  np.concatenate((Y_LS,Y_sen),axis = 0)
print(Y_train.shape)

Y_val=  np.concatenate((Y_val_LandSAT,Y_val_sen),axis = 0)
print(Y_val.shape)

Y_test=  np.concatenate((Y_test_LandSAT,Y_test_sen),axis = 0)
print(Y_test.shape)

In [ ]:
# shuffle the training data

In [ ]:
X, Y = shuffle(X_train, Y_train, random_state=0)

In [ ]:
plt.imshow(X[0]+Y[0,...,np.newaxis])

In [ ]:
plt.imshow(X_val[1002]+Y_val[1002,...,np.newaxis])

In [ ]:
plt.imshow(X_test[1000]+Y_test[1000,...,np.newaxis])

# Pytorch DataLoaders

In [ ]:
class NDWIDataset(Dataset):

    def __init__(self, images, masks, transform=None, test_transform=None):
        self.images     = images
        self.masks      = masks
        self.transforms = transform

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        
        image = self.images[idx]
        mask = self.masks[idx]

        if self.transforms:
            augmentations = self.transforms(image=image, mask=mask)
        
        image = augmentations['image']
        mask  = augmentations['mask']
        mask  = mask[np.newaxis,:,:]
        
        return [image.type(torch.float32), 
                mask.type(torch.float32)]
        

In [ ]:
class NDWIDataset_pseudo(Dataset):

    def __init__(self, imgs, model_pred, threshold=None, transform=None, test_transform=None):
        self.image   = imgs
        self.model   = model_pred
        self.transforms = transform
        self.threshold  = threshold

    def __len__(self):
        return len(self.image)
    
    def __getitem__(self, idx):
        image = self.image[idx].astype('float32')
        model = self.model
        image= image[...,2::-1].copy()
        image= image - image.min(axis=(0,1))
        image= image / image.max(axis=(0,1))
        
        if self.transforms:
            augmentations = self.transforms(image=image)
        
        image = augmentations['image']
        mask  = ((model(image[np.newaxis].cuda()).cpu().detach())>self.threshold)*1.
        mask  = mask.squeeze()
        mask  = mask[np.newaxis,:,:]
        
        return [image.type(torch.float32), 
                mask.type(torch.float32)]
        

In [ ]:
class NDWIDataset_from_dir(Dataset):

    def __init__(self, img_dir, model_pred, threshold=None, transform=None, test_transform=None):
        self.dir     = img_dir
        self.model   = model_pred
        self.transforms = transform
        self.threshold  = threshold

    def __len__(self):
        return len(self.dir)
    
    def __getitem__(self, idx):
        path = self.dir[idx]
        model = self.model
        image= (tiff.imread(path)).astype('float32')
        image= image[...,2::-1].copy()
        image= image - image.min(axis=(0,1))
        image= image / image.max(axis=(0,1))
        
        if self.transforms:
            augmentations = self.transforms(image=image)
        
        image = augmentations['image']
        mask  = ((model(image[np.newaxis].cuda()).cpu().detach())>self.threshold)*1.
        mask  = mask.squeeze()
        mask  = mask[np.newaxis,:,:]
        
        return [image.type(torch.float32), 
                mask.type(torch.float32)]

In [ ]:
class NDWIDataset_from_dir_reg(Dataset):

    def __init__(self, img_dir, transform=None, test_transform=None):
        self.dir     = img_dir
        self.transforms = transform

    def __len__(self):
        return len(self.dir)
    
    def __getitem__(self, idx):
        eps  = 1e-7
        path = self.dir[idx]
        image= (tiff.imread(path)).astype('float32')
        mask = (((image[...,1]-image[...,3] +eps)/(image[...,1]+image[...,3] +eps))>0).astype('float32')
        mask = binary_dilation(mask)-mask
        
        image= image[...,2::-1].copy()
        image= (image) - (image.min(axis=(0,1)))
        image= (image+eps) / (image.max(axis=(0,1))+eps)
        
        if self.transforms:
            augmentations = self.transforms(image=image, mask=mask)
        
        image = augmentations['image']
        mask = augmentations['mask']
        mask = mask[np.newaxis,:,:]
        
        
        return [image.type(torch.float32), 
                mask.type(torch.float32)]        

In [ ]:
def data(trans, trans_test, X_train, Y_train, X_val, Y_val, X_test, Y_test, split=0.9, val=True, batch_size=16):
    torch.manual_seed(49)
    random.seed(49)
    trainset= NDWIDataset(X_train, Y_train, transform=trans)

    if val:
        print(f'Training:{len(X_train)}, Validation:{len(X_val)}')
        print(f'Testing: {len(X_test)}')
        
        valset  = NDWIDataset(X_val, Y_val, transform=trans_test)
        testset = NDWIDataset(X_test, Y_test, transform=trans_test)
        image_datasets = {'train': trainset, 'val': valset, 'test': testset}
        batch_size = batch_size

        dataloaders = {
          'train': DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory = True),#, num_workers=8),
          'val': DataLoader(valset, batch_size=batch_size, shuffle=True, pin_memory = True),#, num_workers=8),
          'test': DataLoader(testset, batch_size=batch_size, shuffle=False, pin_memory = True)#, num_workers=8)
        }
        
    else:
        print(f'Training:{len(X_train)}')
        print(f'Testing: {len(X_test)}')
        testset = NDWIDataset(X_test, Y_test, transform=trans_test)
        image_datasets = {'train': trainset, 'test': testset}
        batch_size = batch_size

        dataloaders = {
          'train': DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory = True),#, num_workers=8),
          'test': DataLoader(testset, batch_size=batch_size, shuffle=False, pin_memory = True)#, num_workers=8)
        }
        
    
    return dataloaders

In [ ]:
def data_from_dir_reg(trans, trans_test, list_train, list_val, list_test, split=0.9, factor=1, val=True, batch_size=16):
    torch.manual_seed(49)
    random.seed(49)
    trainset = NDWIDataset_from_dir_reg(list_train, transform=trans)

    if val:
#         lengths = [int(len(dataset)*split), int(len(dataset)*(1-split))+factor]
        
        valset  = NDWIDataset_from_dir_reg(list_val, transform=trans)
        testset = NDWIDataset_from_dir_reg(list_test, transform=trans_test)
        print(f'Training:{len(list_train)}, Validation:{len(list_val)}')
        print(f'Testing: {len(list_test)}')
        image_datasets = {'train': trainset, 'val': valset, 'test': testset}
        batch_size = batch_size

        dataloaders = {
          'train': DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory = True),#, num_workers=8),
          'val': DataLoader(valset, batch_size=batch_size, shuffle=True, pin_memory = True),#, num_workers=8),
          'test': DataLoader(testset, batch_size=batch_size, shuffle=False, pin_memory = True)#, num_workers=8)
        }
        
    else:
        lengths = [int(len(dataset))]
        print(f'Training:{len(list_train)}')
        print(f'Testing: {len(list_test)}')
        testset = NDWIDataset_from_dir_reg(list_test, transform=trans_test)
        image_datasets = {'train': dataset, 'test': testset}
        batch_size = batch_size

        dataloaders = {
          'train': DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory = True),#, num_workers=8),
          'test': DataLoader(testset, batch_size=batch_size, shuffle=False, pin_memory = True)#, num_workers=8)
        }
        
    
    return dataloaders

In [ ]:
def data_pseudo(trans, img_paths, model, batch_size=32, split=None):
    torch.manual_seed(49)
    random.seed(49)
    if split==None:
        dataset = NDWIDataset_from_dir(img_paths, model, transform=trans)
        lengths = [int(len(dataset))]
        print(f'Pseudo-Training:{lengths[0]}')
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory = True)#, num_workers=8)
    else:
        dataset = NDWIDataset_from_dir(img_paths, model, transform=trans)
        lengths = [int(len(dataset)*split), int(len(dataset)*(1-split))+1]
        trainset, valset = torch.utils.data.random_split(dataset, lengths)
        
        print(f'Pseudo-Training, Train: {len(trainset)}, Validation: {len(valset)}')
        dataloader = {'train':DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory = True),
                      'val'  :DataLoader(valset, batch_size=batch_size, shuffle=True, pin_memory = True)}

    
    return dataloader

In [ ]:
trans = A.Compose([
#     A.RandomCrop(128,128),
#     A.VerticalFlip(),
#     A.HorizontalFlip(),
    ToTensorV2()])
trans_test = A.Compose([
#              A.RandomCrop(128,128),
             ToTensorV2()])

In [ ]:
def plot_rand(dataloader, set_='train'):
    if set_==None:
        for x,y in dataloader:
            x,y = x.numpy().transpose([0,2,3,1]), y.numpy().squeeze()
            break
    else:
        for x,y in dataloader[set_]:
            x,y = x.numpy().transpose([0,2,3,1]), y.numpy().squeeze()
            break
    rand = np.random.randint(0,x.shape[0])
    plt.subplot(121)
    plt.imshow(x[rand])
    plt.title('Image')
    plt.axis('off')
    plt.subplot(122)
    plt.imshow(y[rand])
    plt.title('Contour')
    plt.axis('off')
    plt.show()
#     return x,y

In [ ]:
# dataloaders  = data_from_dir_reg(trans, trans_test, list_train, list_val, list_test, split=0.9)
# # dataloader_p = data_pseudo(trans, list_paths, model_pred)

# Evaluation Metrics

In [ ]:
def f1_score(y_pred, y_true, threshold=0.5):
    
    y_true = y_true.view(-1)
    y_pred = y_pred.view(-1)
    y_pred = ((y_pred>threshold)*1.).type(torch.float32)
    
    tp = torch.sum(y_true*(y_pred))
    tn = torch.sum((1-y_true)*((1-y_pred)))
    fp = torch.sum((1-y_true)*(y_pred))
    fn = torch.sum((y_true)*((1-y_pred)))
    
    pr  = ((tp+1.)/(tp+fp+1.))
    rec = ((tp+1.)/(tp+fn+1.))
    f1  = ((2*pr*rec)/(pr+rec))
    return f1

 Exponential MA = $V_{t} = \beta V_{t-1} + (1-\beta)(current)$

In [ ]:
import time
def train_round1(model, dataloaders, loss_fn, optimizer, acc_fn, random_state=49, epochs=1):
    
    
    torch.manual_seed(random_state)
    np.random.seed(random_state)
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    start = time.time()                                        #Initialize time to calculate time it takes to train model
    model.to(device)                                               #Move model to GPU     

    counter=0
    train_loss, valid_loss = [], []                            #Running training and validation loss
    val_epoch, f1_epoch = [0],[0]
    loss_list = []
    times     = []
    
    for epoch in range(epochs):
        start_epoch = time.time()
        print(f'Epoch {epoch}')
        print(scheduler.get_last_lr())
    

    #########################################Begin Model Training######################################################
    ###################################################################################################################
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()                             # Set training mode all the layers including batchnorm and dropout work in this
                dataloader = dataloaders['train']         #get the training data
            else:
                model.eval()                              # Set model to evaluate mode deactivates the batchnorm and dropout layers
                dataloader = dataloaders['val']           #get the validation data

            running_loss = 0.0                            #running loss to be used for visualization later
            step = 0                                      #Batch number
            
            if phase == 'train':  
                f1 = []
                AP = []
                for inputs, labels in dataloader:
                    x, y = inputs.to(device), labels.to(device)
                    step += 1

                    optimizer.zero_grad()                                   # zero the gradients
                    outputs = model(x)                                      #get model output for a given input
                    
                    
                     # switching the values of 0 and 1 to create a negative image
                    negative = torch.where(y==0,torch.tensor(-1.0).to(device),y).to(device)
                    negative = torch.where(negative==1,torch.tensor(0.0).to(device),negative).to(device)
                    negative = torch.where(negative== -1.0,torch.tensor(1.0).to(device),negative).to(device)
                    
                    
                    #################Metrics###################
                    f1.append(acc_fn(outputs, y).cpu().detach().numpy())
                    AP.append(average_precision_score(y.reshape(-1).cpu().detach().numpy(),  outputs.reshape(-1).cpu().detach().numpy()))

                    ############################################

                    ##################Calculate Loss, backprop, and update###############
                    

                    tripletLoss = triplet_Loss(outputs, y,negative)
                    
                    loss           = loss_fn(outputs, y) + tripletLoss
                    
#                     print(tripletLoss)
                    
                    train_loss.append(loss.cpu().detach().numpy())
                    loss.backward()
                    optimizer.step()
                    print(f'Current step: {step}, AllocMem (Mb): {torch.cuda.memory_allocated()/1024/1024:.3f}, Loss: {loss:.3f}, F1: {np.mean(f1):.3f},  AP: {np.mean(AP):.3f}', end='\r') 
                    ######################################################################
        
            else:  
                loss_val = []
                f1=[]
                AP = []
                with torch.no_grad():
                    for inputs, labels in dataloader:
                        x, y = inputs.to(device), labels.to(device)
                        optimizer.zero_grad()                                   # zero the gradients
                        outputs = model(x)                                      #get model output for a given input

                        #################Metrics###################
                        f1.append(acc_fn(outputs, y).cpu().detach().numpy())
                        AP.append(average_precision_score(y.reshape(-1).cpu().detach().numpy(),  outputs.reshape(-1).cpu().detach().numpy()))

                    ############################################
                        vlCurrent = loss_fn(outputs, y)
                        ##################Calculate Loss, backprop, and update###############
                        valid_loss.append(vlCurrent.cpu().detach().numpy())
                        loss_val.append(valid_loss[-1])
                val_epoch.append(np.mean(loss_val))
                f1_epoch.append(np.mean(f1))
                print()
                print()
                print(f' Loss val: {val_epoch[-1]:.3f}, F-Score val:{f1_epoch[-1]:.3f}, AP val:{AP[-1]:.3f} \n') 
                ######################################################################
                

            print()
            time_elapsed = time.time() - start_epoch
            times.append(time_elapsed)
            print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
            print('-' * 10)      

        scheduler.step()
        torch.save(model, save_last+ '\\' + f'Epoch_{str(epoch).zfill(3)}'+ '.pth')
        epoch+=1
    #########################################End Model Training######################################################
    ###################################################################################################################
    
    #Total training time including time to test
    time_elapsed = time.time() - start
    print('\n Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
    
#     torch.save(model, save_last+ '\\' + f'Feature32_{str(1000)}_images'+ '.pth')
    
    return {'Train Loss':train_loss,
            'Valid Loss':valid_loss,
            'Times'     :times,
            'f1_epoch':f1_epoch,
            'Epochs': epoch}

## Round1(Train Using Labeled Data Only)

In [ ]:
X_train_labeled = X#[:1000]
Y_train_labeled = Y#[:1000]

print(X_train_labeled.shape)

In [ ]:
trans = A.Compose([
#    A.VerticalFlip(),
#    A.HorizontalFlip(),    
    ToTensorV2()])
trans_test = A.Compose([
                ToTensorV2()])


In [ ]:
testModel = r"C:\Users\HPCL\OneDrive - University of New Orleans (1)\Alsamman Research\HPC_Results\Saiful_Results_1\ComparisonsTests\DeepLabBest.pth"

In [ ]:


# model = UNET_multiscale2().to(device)


opt = optim.Adam(model.parameters(), lr=0.003)
scheduler = optim.lr_scheduler.MultiStepLR(opt, milestones=[100,180], gamma=0.1)

summary(testModel,(3,128,128))


In [ ]:
# with open(r'{}historyFeature32_trainLoss.txt'.format(save_last), 'w') as f:
#     print(history['Valid Loss'], file=f)

In [ ]:
def test(model, dataloaders, loss_fn, acc_fn, random_state=49, epochs=1):
    
    
    torch.manual_seed(random_state)
    np.random.seed(random_state)
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    start = time.time()                                        #Initialize time to calculate time it takes to train model
    model.to(device)                                               #Move model to GPU     

    model.eval()
    
    counter=0
    #train_loss,
    test_loss = []                         #Running training and validation loss
    loss_epoch, f1_epoch = [0],[0]
    AP_epoch =[0]
    loss_list = []
    times     = []
    
    for epoch in range(epochs):
        start_epoch = time.time()
        print(f'Epoch {epoch}')
        #print(scheduler.get_last_lr())
    

 ################ MODEL TESTING  #############################

        dataloader = dataloaders['test']         #get the training data

        step = 0
        loss_test = []
        f1=[]
        AP = []
        with torch.no_grad():
            for inputs, labels in dataloader:
                x, y = inputs.to(device), labels.to(device)
#                 print(y.shape)
                #optimizer.zero_grad()                                   # zero the gradients
                outputs= model(x)                                      #get model output for a given input
                
#                 print(outputs.shape)
                #################Metrics###################
                f1.append(acc_fn(outputs, y).cpu().detach().numpy())
                AP.append(average_precision_score(y.reshape(-1).cpu().detach().numpy(),  outputs.reshape(-1).cpu().detach().numpy()))
                lossTest, mLoss = loss_fn(outputs, y)
                
                loss_test.append(lossTest.cpu().detach().numpy())
            ############################################
                print(f'Current step: {step}, AllocMem (Mb): {torch.cuda.memory_allocated()/1024/1024:.3f},  F1: {np.mean(f1):.3f},  AP: {np.mean(AP):.3f}') 

                step+=1
                ##################Calculate Loss, backprop, and update###############
               
                test_loss.append(loss_test[-1])
        loss_epoch.append(np.mean(loss_test))
        f1_epoch.append(np.mean(f1))
        AP_epoch.append(np.mean(AP))
        print()
        print()
        print(f' Loss test: {loss_epoch[-1]:.3f}, F-Score test:{f1_epoch[-1]:.3f}, AP val:{AP_epoch[-1]:.3f} \n') 
        ######################################################################
                

            

        #scheduler.step()
#         torch.save(model, save_last+ '\\' + f'Epcoh_{str(epoch).zfill(3)}'+ '.pth')
        epoch+=1
    #########################################End Model Training######################################################
    ###################################################################################################################
    
    #Total training time including time to test
    time_elapsed = time.time() - start
    print('\n Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
    
#     torch.save(model, save_last+ '\\' + f'Feature32_{str(1000)}_images'+ '.pth')
    
    return {
            'f1_epoch':f1_epoch,
            'Epochs': epoch}

In [ ]:
batch_size = 32
dataloaders = data(trans, trans_test, X_train_labeled, Y_train_labeled, X_val, Y_val, X_test, Y_test, batch_size=batch_size)

In [ ]:
save_last

In [ ]:
# history = train_round1(model, dataloaders, loss_fn = border_loss, optimizer = opt, acc_fn = f1_score, epochs=50)

In [ ]:
# df = pd.DataFrame(history['f1_epoch'])
                  
# df.to_csv(save_last+ '\\'+ "UNET_MC_2_Batch16.csv")

In [ ]:
# testModel = torch.load(f"{save_last}\\Epoch_041.pth")

In [ ]:
test_history = test(testModel, dataloaders, loss_fn = border_loss, acc_fn = f1_score,epochs=1)

In [ ]:
dataloaderSentinel = data(trans, trans_test, X_sen, Y_sen, X_val_sen, Y_val_sen, X_test_sen, Y_test_sen, batch_size=batch_size)


dataloaderLandSAT = data(trans, trans_test, X_LS, Y_LS, X_val_LandSAT, Y_val_LandSAT, X_test_LandSAT, Y_test_LandSAT, batch_size=batch_size)




In [ ]:
test_Sen = test(testModel, dataloaderSentinel, loss_fn = border_loss, acc_fn = f1_score, epochs=1)

In [ ]:
test_LS = test(testModel, dataloaderLandSAT, loss_fn = border_loss, acc_fn = f1_score, epochs=1)

In [ ]:
#     return {'Train Loss':train_loss,
#             'Valid Loss':valid_loss,
#             'Times'     :times,
#             'Epochs': epoch}